# Tools  🔨🪚🔩⚙️⚒️🧲🧰🔧🪛🔩⚙️🔦🧭🔗⛓️🧮
도구는 에이전트가 현실 세계에서 '행동'할 수 있게 합니다.
세심한 설명은 에이전트가 도구를 사용하는 방법을 발견하는 데 도움이 될 수 있습니다.

LangChain supports many tool formats and tool sets. Here we will cover some common cases, but check the [docs](https://docs.langchain.com/oss/python/langchain/tools) for more information.

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv("../.env")

# Check and print results
doublecheck_env("../.env")

OPENAI_API_KEY=****VLwA
UPSTAGE_API_KEY=****d51g
LANGCHAIN_TRACING_V2=false
LANGCHAIN_ENDPOINT=****.com
LANGCHAIN_PROJECT=****demy
LANGCHAIN_API_KEY=****fc10
PINECONE_API_KEY=****4e73
HUGGINGFACE_API_KEY=****aRMB
TAVILY_API_KEY=****2UUC
SERPER_API_KEY=****e4cd
WOLFRAM_ALPHA_APPID=****LT74
POLYGON_API_KEY=****8Gso


## Calculator example

아래 예제에서 LLM은 도구 호출 시기와 방법을 결정하기 위해 문서 문자열과 추론된 인수 및 인수 유형을 활용합니다.

In [2]:
from typing import Literal

from langchain.tools import tool


@tool
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """Perform basic arithmetic operations on two real numbers."""
    print("🧮 Invoking calculator tool")
    # Perform the specified operation
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero is not allowed.")
        return a / b
    else:
        raise ValueError(f"Invalid operation: {operation}")

In [3]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-5",
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

이제 에이전트는 도구를 사용할 것입니다.

In [4]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "what is 3.1125 * 4.1234"}]}
)
print(result["messages"][-1].content)

12.8340825


We can check the **metadata** in [LangSmith Observability](https://smith.langchain.com/public/b77bde6c-f0ad-4256-bfab-7d514ece3405/r) to see this.

도구 설명은 큰 영향을 미칠 수 있습니다.
아래 예제에서 입력값이 정수이므로 계산기 도구가 호출되지 않을 수 있습니다.
(실행마다 결과가 다를 수 있음)

In [5]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3 * 4"}]})
print(result["messages"][-1].content)

12


입력이 실수임에도 불구하고 이 도구가 호출되지 않는 경우가 종종 있습니다. (실행마다 결과가 다릅니다)

In [6]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3.0 * 4.0"}]})
print(result["messages"][-1].content)

12.0


## Adding a more detailed description
기본 설명으로도 충분한 경우가 많지만, LangChain은 향상된 설명 기능을 지원합니다. 
아래 예시는 한 가지 방법인 Google 스타일 인자 설명을 사용합니다. `parse_docstring=True`와 함께 사용하면 인자 설명을 파싱하여 모델에 전달합니다. 
도구의 이름을 변경하고 설명을 수정할 수 있습니다. 
표준 도구를 공유하지만 에이전트별 지침을 원하는 경우 효과적입니다.

In [7]:
from typing import Literal

from langchain.tools import tool


@tool(
    "calculator",
    parse_docstring=True,
    description=(
        "Perform basic arithmetic operations on two real numbers."
        "Use this whenever you have operations on any numbers, even if they are integers."
    ),
)
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """Perform basic arithmetic operations on two real numbers.

    Args:
        a (float): The first number.
        b (float): The second number.
        operation (Literal["add", "subtract", "multiply", "divide"]):
            The arithmetic operation to perform.

            - `"add"`: Returns the sum of `a` and `b`.
            - `"subtract"`: Returns the result of `a - b`.
            - `"multiply"`: Returns the product of `a` and `b`.
            - `"divide"`: Returns the result of `a / b`. Raises an error if `b` is zero.

    Returns:
        float: The numerical result of the specified operation.

    Raises:
        ValueError: If an invalid operation is provided or division by zero is attempted.
    """
    print("🧮  Invoking calculator tool")
    # Perform the specified operation
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero is not allowed.")
        return a / b
    else:
        raise ValueError(f"Invalid operation: {operation}")

In [8]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

In [9]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3.0 * 4.0"}]})
print(result["messages"][-1].content)

🧮  Invoking calculator tool
12.0


Let's check our [LangSmith Observability trace](https://smith.langchain.com/public/7d65902c-bd3c-4fc6-bbd3-7c1d66566fda/r) to see the tool description.

In [10]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3 * 4"}]})
print(result["messages"][-1].content)

🧮  Invoking calculator tool
12


## Try your own.
Create a tool of your own and try it here!

In [11]:

@tool
def your_tool(
    a: float, b: float, 
) -> float:
    """Perform your favorite operation

    Args:
        a (float): operator a description
        b (float): operator b description

    Returns:
        float: description
    """
    pass

In [12]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[your_tool],
    system_prompt="You are a helpful assistant",
)